In [33]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from functools import reduce

import numpy as np
from scipy.optimize import minimize

In [34]:
# dij需求i到设施j的成本
d = [[1, 2], [1, 2]]
n = 2   # 两个设施点
m = 2   # 两个需求点
num_qubits = n + 2 * n * m

# gi设施i的建设成本
g = [2, 1]

In [35]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

# 定义σ+和σ-矩阵
sigma_plus = np.array([[0, 1], [0, 0]])
sigma_minus = np.array([[0, 0], [1, 0]])

def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H

def calculate_hamiltonian(v, w):
    n = len(v[0])
    m = len(v)
    hamiltonian = np.zeros((2**n, 2**n))

    for i in range(m):
        term1 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, v[i][j]) for j in range(n)])
        term2 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, w[i][j]) for j in range(n)])
        term3 = reduce(np.kron, [np.linalg.matrix_power(sigma_plus, w[i][j]) for j in range(n)])
        term4 = reduce(np.kron, [np.linalg.matrix_power(sigma_minus, v[i][j]) for j in range(n)])

        hamiltonian += term1 @ term2 + term3 @ term4

    return hamiltonian

def first_nonzero_index(arr, total_bits=3):
    for i, num in enumerate(arr):
        if num != 0:
            binary_repr = format(i, '0' + str(total_bits) + 'b')
            return binary_repr

In [36]:
def generate_Hp(n, m, d, g):
    # 初始化 Hp 矩阵为零矩阵
    Hp = np.zeros((2**num_qubits, 2**num_qubits))
    for i in range(m):
        for j in range(n):
            Hp += d[i][j] * (add_in_target(num_qubits, n * (1 + i) + j) - np.eye(2**num_qubits)) / 2
    
    for j in range(n):
        Hp +=  g[j] * (add_in_target(num_qubits, j)- np.eye(2**num_qubits)) / 2
    return Hp
        
Hp = generate_Hp(n, m, d, g)

In [37]:
import sys
sys.path.append('../../')
import zlibrary.linear_system as ls


In [38]:
def gnrt_gate_h(nonzero_indices):
    result = []
    for row in nonzero_indices:
        filtered_arr = [0 if x == -1 else 1 for x in row if x != 0]
        binary_str = ''.join(map(str, filtered_arr))
        map_num = int(binary_str, 2)
        length = len(row)
        scale = 2**length
        matrix = np.zeros((scale, scale))
        matrix[map_num, scale - 1 - map_num] = 1
        matrix[scale - 1 - map_num, map_num] = 1
        result.append(matrix)
    return result

# print("非零元素的索引：", nonzero_indices)

u = ls.gnrt_u(n, m)
print(u)
u_size = len(u)
nonzero_indices = ls.find_nonzero_indices(u)
gate_hs = gnrt_gate_h(nonzero_indices)
print(nonzero_indices)

[[-1.  1. -1.  1. -1.  1.  0.  0.  0.  0.]
 [ 0.  0.  1. -1.  0.  0. -1.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  1.  0.  1.  0.]
 [ 0.  1. -1.  1.  0.  0.  1.  0.  0.  1.]]
[[0, 1, 2, 3, 4, 5], [2, 3, 6, 7], [0, 6, 8], [1, 2, 3, 6, 9]]


In [39]:
from scipy.linalg import expm
def build_circ(params):
  depth = len(params) // 2
  qc = QuantumCircuit(num_qubits)
  beta = params[:depth]
  gamma = params[depth:]
  for i in [4,6,8]:
    qc.x(i)
  for dp in range(depth):
    qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits))
    for gate_h, ind in zip(gate_hs, nonzero_indices):
      qc.unitary(expm(-1j * gamma[dp] * gate_h), (9 - i for i in ind))
  qc.measure_all()
  return qc

In [40]:
# import sys
# sys.path.append('../../')
# import zlibrary.linear_system as ls
# Cons = np.array([[-1, 0, 1, 0, 0, 0, 1, 0, 0, 0],
#               [-1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
#               [0, -1, 0, 1, 0, 0, 0, 1, 0, 0],
#               [0, -1, 0, 0, 0, 1, 0, 0, 0, 1],
#               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#               [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
#               [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]])
# u = ls.find_basic_solution(Cons)
# v = np.where(u == 1, 1, 0)
# w = np.where(u == -1, 1, 0)
# Hd = calculate_hamiltonian(v, w)
# from scipy.linalg import expm
# def build_circ2(params):
#   depth = len(params) // 2
#   qc = QuantumCircuit(num_qubits)
#   beta = params[:depth]
#   gamma = params[depth:]
#   for i in [4,6,8]:
#     qc.x(i)
#   for dp in range(depth):
#     qc.unitary(expm(-1j * beta[dp] * Hp), range(num_qubits))
#     qc.unitary(expm(-1j * gamma[dp] * Hd), range(num_qubits))
#   qc.measure_all()
#   return qc

In [41]:
# from qiskit import transpile
# simulator = Aer.get_backend('statevector_simulator')
# qc1 = build_circ(np.full(2 * 2, np.pi/3))
# new_circuit1 = transpile(qc1, simulator)
# job1 = simulator.run(new_circuit1)
# print(job1.result().get_statevector())

In [42]:
# qc2 = build_circ2(np.full(2 * 2, np.pi/3))
# # 执行电路并获取最终状态向量
# new_circuit2 = transpile(qc2, simulator)
# job2 = simulator.run(new_circuit2)
# print(job2.result().get_statevector())

In [43]:
# ls.set_print_form(3)
# print(job2.result().get_statevector())

In [44]:
build_circ(np.full(2 * 2, np.pi/3)).draw()

┌──────────┐                                    ┌──────────┐»
    q_0: ─────┤0         ├────────────────────────────────────┤4         ├»
              │          │                        ┌──────────┐│          │»
    q_1: ─────┤1         ├────────────────────────┤2         ├┤          ├»
              │          │            ┌──────────┐│          ││          │»
    q_2: ─────┤2         ├────────────┤3         ├┤          ├┤          ├»
              │          │            │          ││          ││          │»
    q_3: ─────┤3         ├────────────┤2         ├┤1         ├┤3         ├»
         ┌───┐│          │┌──────────┐│          ││          ││          │»
    q_4: ┤ X ├┤4         ├┤5         ├┤          ├┤          ├┤  Unitary ├»
         └───┘│  Unitary ││          ││  Unitary ││          ││          │»
    q_5: ─────┤5         ├┤4         ├┤          ├┤  Unitary ├┤          ├»
         ┌───┐│          ││          ││          ││          ││          │»
    q_6: ┤ X ├┤6         ├┤3         ├┤1         ├┤          ├┤2         ├»
         └───┘│          ││  Unitary ││          ││          ││          │»
    q_7: ─────┤7         ├┤2         ├┤0         ├┤          ├┤1         ├»
         ┌───┐│          ││          │└──────────┘│          ││          │»
    q_8: ┤ X ├┤8         ├┤1         ├────────────┤          ├┤0         ├»
         └───┘│          ││          │            │          │└──────────┘»
    q_9: ─────┤9         ├┤0         ├────────────┤0         ├────────────»
              └──────────┘└──────────┘            └──────────┘            »
meas: 10/═════════════════════════════════════════════════════════════════»
                                                                          »
«         ┌──────────┐                                    ┌──────────┐ ░ ┌─┐   »
«    q_0: ┤0         ├────────────────────────────────────┤4         ├─░─┤M├───»
«         │          │                        ┌──────────┐│          │ ░ └╥┘┌─┐»
«    q_1: ┤1         ├────────────────────────┤2         ├┤          ├─░──╫─┤M├»
«         │          │            ┌──────────┐│          ││          │ ░  ║ └╥┘»
«    q_2: ┤2         ├────────────┤3         ├┤          ├┤          ├─░──╫──╫─»
«         │          │            │          ││          ││          │ ░  ║  ║ »
«    q_3: ┤3         ├────────────┤2         ├┤1         ├┤3         ├─░──╫──╫─»
«         │          │┌──────────┐│          ││          ││          │ ░  ║  ║ »
«    q_4: ┤4         ├┤5         ├┤          ├┤          ├┤  Unitary ├─░──╫──╫─»
«         │  Unitary ││          ││  Unitary ││          ││          │ ░  ║  ║ »
«    q_5: ┤5         ├┤4         ├┤          ├┤  Unitary ├┤          ├─░──╫──╫─»
«         │          ││          ││          ││          ││          │ ░  ║  ║ »
«    q_6: ┤6         ├┤3         ├┤1         ├┤          ├┤2         ├─░──╫──╫─»
«         │          ││  Unitary ││          ││          ││          │ ░  ║  ║ »
«    q_7: ┤7         ├┤2         ├┤0         ├┤          ├┤1         ├─░──╫──╫─»
«         │          ││          │└──────────┘│          ││          │ ░  ║  ║ »
«    q_8: ┤8         ├┤1         ├────────────┤          ├┤0         ├─░──╫──╫─»
«         │          ││          │            │          │└──────────┘ ░  ║  ║ »
«    q_9: ┤9         ├┤0         ├────────────┤0         ├─────────────░──╫──╫─»
«         └──────────┘└──────────┘            └──────────┘             ░  ║  ║ »
«meas: 10/════════════════════════════════════════════════════════════════╩══╩═»
«                                                                         0  1 »
«                                 
«    q_0: ────────────────────────
«                                 
«    q_1: ────────────────────────
«         ┌─┐                     
«    q_2: ┤M├─────────────────────
«         └╥┘┌─┐                  
«    q_3: ─╫─┤M├──────────────────
«          ║ └╥┘┌─┐               
«    q_4: ─╫──╫─┤M├───────────────
«          ║  ║ └╥┘┌─┐            
«    q_5: ─╫──╫──╫

In [45]:
def cost_function(x):
  num = [int(char) for char in x]
  C = 0
  for i in range(m):
    for j in range(n):
      C += d[i][j] * num[n * (1 + i) + j]
      
  for j in range(n):
    C += g[j] * num[j]
  return C

def compute_expectation(counts):
  EV = 0
  total_count = 0
  for x, count in counts.items():
    C = cost_function(x)
    EV += C*count
    total_count += count

  return EV/total_count


def expectation_from_sample(shots = 2000):
  backend = Aer.get_backend('qasm_simulator')
  backend.shots = shots

  def execute_circ(theta):
    qc = build_circ(theta)
    counts = backend.run(qc, seed_simulator=10, shots=shots).result().get_counts()
    return compute_expectation(counts)
  
  return execute_circ

In [46]:
from numpy.lib.utils import source
from scipy.optimize import minimize
import numpy as np
# 初始化迭代计数器
iteration_count = 0
def test(par):
  global iteration_count
  iteration_count = 0
  expectation = expectation_from_sample()
  def callback(x):
      global iteration_count
      iteration_count += 1
      if iteration_count % 10 == 0:
          print(f"Iteration {iteration_count}, Result: {expectation(x)}")
  # 设定最大迭代次数
  max_iterations = 1000

  # 使用 COBYLA 方法进行最小化，并设置 callback 函数
  res = minimize(expectation, par, method='COBYLA', options={'maxiter': max_iterations}, callback=callback)
  # 输出最终结果
  print("Final Result:", res)
  backend = Aer.get_backend('aer_simulator')
  backend.shots = 100000

  shots=100000
  qc_res = build_circ(params=res.x)

  counts = backend.run(qc_res, seed_simulator=10, shots = shots).result().get_counts()
  # plot_histogram(counts)
  sorted_counts = sorted(counts, key=counts.get, reverse=True)
  print("\n----------------- Full result ---------------------")
  print("selection\t\tprobability\tvalue")
  print("---------------------------------------------------")
  for x in sorted_counts[:20]:
    print(x, "{:.2f}%".format(counts[x] / shots * 100), cost_function(x))

In [47]:
for dep in range(1,4):
  print(f'depth == {dep}')
  test(np.full(dep * 2, np.pi/3))
  print()

depth == 1


Iteration 10, Result: 5.0
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 5.0
       x: [ 1.047e+00  1.047e+00]
    nfev: 13
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 100.00% 5

depth == 2
Iteration 10, Result: 5.0
Iteration 20, Result: 5.0
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 5.0
       x: [ 1.047e+00  1.047e+00  1.047e+00  1.047e+00]
    nfev: 25
   maxcv: 0.0

----------------- Full result ---------------------
selection		probability	value
---------------------------------------------------
0101010000 100.00% 5

depth == 3
Iteration 10, Result: 5.0
Iteration 20, Result: 5.0
Iteration 30, Result: 5.0
Final Result:  message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 5.0
       x: [ 1.047e+00  1.047e+00  1.047e+00  1.047e+00  1